<a href="https://colab.research.google.com/github/ayesha-anjum-shaik/PySparkCrashCourse/blob/main/CourseProjectPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

challenge.csv  spark-3.1.1-bin-hadoop3.2
sample_data    spark-3.1.1-bin-hadoop3.2.tgz


In [2]:
# read the input csv file with header and store in data frame
df = spark.read.format("csv").option("header","true").load("challenge.csv")
df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [3]:
# add a column to say yes or no whether the company is Mexico
from pyspark.sql.functions import *
df = df.withColumn("Is it Mexico",when(df.Country == 'Mexico','Yes').otherwise('No'))
df.show()

+---------------+--------------+-----------------+----------+------------+
|     ip_address|       Country|      Domain Name|Bytes_used|Is it Mexico|
+---------------+--------------+-----------------+----------+------------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|          No|
| 119.239.207.13|         China|         youtu.be|        51|          No|
|  68.69.217.210|         China|        adobe.com|        10|          No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|          No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|          No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|          No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|          No|
| 127.242.24.138|         China| surveymonkey.com|       123|          No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|          No|
|   237.54.11.63|         China|       amazon.com|        83|          No|
| 252.141.157.25|        

In [5]:
# group by new column and sum of bytes used
import pyspark.sql.functions as sqlfunc
df1 = df.groupBy('Is it Mexico').agg(sqlfunc.sum('Bytes_used').alias('Mexico_Bytes'))
df1.show()

+------------+------------+
|Is it Mexico|Mexico_Bytes|
+------------+------------+
|          No|    508076.0|
|         Yes|      6293.0|
+------------+------------+



In [7]:
# group by country and use the sqlfunc.countDistinct function to calculate the no. of ip addresses seen in each country
df2 = df.groupBy('Country').agg(sqlfunc.countDistinct('ip_address').alias('no_of_ip_addr'))
df2.show()

+-----------+-------------+
|    Country|no_of_ip_addr|
+-----------+-------------+
|       Chad|            1|
|     Russia|           56|
|   Paraguay|            1|
|      Yemen|            1|
|     Sweden|           28|
|Philippines|           65|
|   Malaysia|            5|
|     Turkey|            1|
|     Malawi|            2|
|    Germany|            5|
|    Comoros|            1|
|Afghanistan|            5|
|     Rwanda|            1|
|      Sudan|            1|
|     France|           21|
|     Greece|            8|
|  Sri Lanka|            3|
|   Dominica|            1|
|  Argentina|           14|
|    Belgium|            1|
+-----------+-------------+
only showing top 20 rows

